In [1]:
import sys
import os
import numpy as np


### Add src directory

In [2]:
# current path is /Users/axhome/AX/MASTER/Courses/DD2424/DD2424-DeepLearning/Assignment1/notebooks
# we want to add the path /Users/axhome/AX/MASTER/Courses/DD2424/DD2424-DeepLearning/Assignment1/src

# add the path to the sys.path
# sys.path.append('/Users/axhome/AX/MASTER/Courses/DD2424/DD2424-DeepLearning/Assignment1/src')
# or use os.path.abspath
sys.path.append(os.path.abspath("../src"))


### Import functions from src files

In [3]:
from data_loader import LoadBatch
from preprocessing import reshape_images, normalize_images


# Exercise - Training a multi-linear classifier

## Task 1 
**Write a function that reads in the data from a CIFAR-10 batch file and returns the image and label data in separate files.**

In [4]:
X, Y, y = LoadBatch(1)
print(X.shape)

(3072, 10000)


In [5]:
print(Y[:, 0])

[0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]


In [6]:
print(y[0])

[6 9 9 ... 1 1 5]


#### Read in Train, Validation and Test data

In [7]:
X_train_raw, Y_train, y_train = LoadBatch(1)
X_val_raw, Y_val, y_val = LoadBatch(2)
X_test_raw, Y_test, y_test = LoadBatch("test_batch")

## Task 2
**pre-process the raw input data**

Compute the mean and standard deviation vector for the
training data and then normalize the training, validation and test data
w.r.t. the training mean and standard deviation vectors.

In [8]:
# transform training data to have zero mean

X_train_mean = np.mean(X_train_raw, axis=1).reshape(-1, 1) # (3072, 1)
X_train_std = np.std(X_train_raw, axis=1).reshape(-1, 1) # (3072, 1)

In [9]:
# normalize the data

# Training data
X_train = X_train_raw - X_train_mean
X_train = X_train / X_train_std

# Validation data
X_val = X_val_raw - X_train_mean
X_val = X_val / X_train_std

# Testing data
X_test = X_test_raw - X_train_mean
X_test = X_test / X_train_std



In [10]:
y_train.shape

(1, 10000)

## Task 3
**Initialise the parameters of the model W (Kxd) and b (Kx1)**

In [11]:
# Random number generation
rng = np.random.default_rng()

# Get the BitGenerator used by deault_rng
#PCG64
BitGen = type(rng.bit_generator)

# use the state from a fresh bit generator 
seed = 42

# dimensions
K = 10 
d = 3072

# set the state of the bit generator
rng.bit_generator.state = BitGen(seed).state

# initialize the network
init_net = {}
init_net["W"] = 0.01*rng.standard_normal(size = (K, d))
init_net["b"] = np.zeros((K, 1))

## Task 4
**Write a function that applies the network function, (i.e. equations 1 and 2) to multiple images and returns the results**

$$
\mathbf{s} = \mathbf{W} \mathbf{x} + \mathbf{b} \quad (1)
$$

$$
\mathbf{p} = \text{SOFTMAX}(\mathbf{s}) \quad (2)
$$


In [12]:
# write function P = ApplyNetwork(X, net)
def ApplyNetwork(X, net):
    W = net["W"]
    b = net["b"]
    P = W @ X + b
    return P


In [13]:
P = ApplyNetwork(X_train[:, 0:100], init_net)

In [ ]:
P.shape


(1, 10000)

## Task 5 
**Write the function that computes the loss function given by equation (5) for a set of images**

$$
J(\mathcal{D}, \lambda, \mathbf{W}, \mathbf{b}) =
\frac{1}{|\mathcal{D}|} \sum_{(\mathbf{x}, \mathbf{y}) \in \mathcal{D}}
l_{\text{cross}} (\mathbf{x}, \mathbf{y}, \mathbf{W}, \mathbf{b})
+ \lambda \sum_{i,j} W_{ij}^{2} \quad \quad \quad (5)
$$


In [15]:
# L is a scalar corresponding to the mean cross-entropy loss of the network’s predictions relative to the ground truth labels.
# P is a matrix containing the network’s predicted class probabilities for each example.
# y is a vector containing the ground truth labels.

def ComputeLoss(P, y):
    # N is the number of examples
    N = P.shape[1]
    print(N)
    print(y.shape)
    # Cross-entropy loss
    loss = -np.log(P[y, np.arange(N)])

    # Mean cross-entropy loss
    loss = np.mean(loss)
    
    return loss



In [ ]:
print(P.min(), P.max())
print(y_train.shape)

# this may be a problem

-2.0640553180667855 2.3586982079988648
(1, 10000)


In [18]:
# note that y_train is of shape (1, 10000)

L = ComputeLoss(P, y_train[0, 0:100].flatten())

100
(100,)


/var/folders/m7/n_7wlxws2c78dvvqtvvsykl40000gn/T/ipykernel_13037/4283909707.py:11: RuntimeWarning: invalid value encountered in log
  loss = -np.log(P[y, np.arange(N)])
